### 予測精度が悪いレコードは、どのようなレコードか検証する

In [156]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib
from sklearn.model_selection import KFold

###訓練データの読み込み
house_age = pd.read_csv('../data/handmade_data/house_age.csv')
area_size = pd.read_csv('../data/handmade_data/area_size.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
contract_span = pd.read_csv('../data/handmade_data/contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('../data/handmade_data/contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('../data/handmade_data/reg_rent.csv')
floor_scores = pd.read_csv('../data/handmade_data/floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('../data/handmade_data/floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('../data/handmade_data/capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('../data/handmade_data/capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('../data/handmade_data/stations.csv')
minits = pd.read_csv('../data/handmade_data/minits.csv')
addresses = pd.read_csv('../data/handmade_data/addresses.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
buildings = pd.read_csv('../data/handmade_data/buildings.csv')
land_aveprice = pd.read_csv('../data/handmade_data/land_aveprice.csv')
land_aveprice.columns = ['平均地価']


rent = pd.read_csv('../data/handmade_data/rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('../data/test_data/test_house_age.csv')
test_area_size = pd.read_csv('../data/test_data/test_area_size.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_contract_span = pd.read_csv('../data/test_data/test_contract_span.csv')
test_contract_span2 = pd.read_csv('../data/test_data/test_contract_span2.csv')
test_reg_rent = pd.read_csv('../data/test_data/test_reg_rent.csv')
test_floor_scores = pd.read_csv('../data/test_data/test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('../data/test_data/test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('../data/test_data/test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('../data/test_data/test_capital_floor_scores2.csv')
test_stations = pd.read_csv('../data/test_data/test_stations.csv')
test_minits = pd.read_csv('../data/test_data/test_minits.csv')
test_addresses = pd.read_csv('../data/test_data/test_addresses.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_buildings = pd.read_csv('../data/test_data/test_buildings.csv')
test_land_aveprice = pd.read_csv('../data/test_data/test_land_aveprice.csv')
test_land_aveprice.columns = ['平均地価']

X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, land_aveprice, addresses, room_arrange[["部屋数","L","D","S"]]], axis=1)
y_train = rent
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_land_aveprice, test_addresses, test_room_arrange[["部屋数","L","D","S"]]], axis=1)
category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','S']

In [121]:
#予測精度の悪いレコード発見のために、訓練データの一部を検証データとする。そして残った訓練データで交差検証を行う。
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
cv = KFold(n_splits=5, shuffle=True, random_state=0)

#これより交差検証のためにインデックスを振り直すので、最後にインデックスを元に戻すことに備えて元々のインデックスを退避させておく。
origin_index = X_train.index #origin_index = y_train.indexでも同じこと
X_train = X_train.reset_index()#交差検証のためにインデックス振り直し
X_train = X_train.drop('index', axis=1)##reset_index()をした結果、元のインデックスが"index"列として含まれてしまうので落としておく
y_train = y_train.reset_index()#交差検証のためにインデックス振り直し
y_train = y_train.drop('index',axis=1)##reset_index()をした結果、元のインデックスが"index"列として含まれてしまうので落としておく

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    params = {
        'objective':'regression',
        'boosting_type':'gbdt',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda': 2.7630799986965487e-05,
        'max_bin': 392,
        'num_leaves': 119
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=1000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
df_importance = pd.DataFrame(importance)
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(importance)

y_valid_pred = model.predict(X_valid, num_iteration=model.best_iteration)

# 以下は、検証データを用いて予測した値と「賃料」とのRSMEを、その他データとともに格納する部分である。
y_valid = list(y_valid['賃料'])
y_valid_pred = list(y_valid_pred)
X_valid_records = X_valid.copy()
X_valid_records = X_valid_records.reset_index()
rmses = []
for (s,t) in zip(y_valid, y_valid_pred):
    rmse_score = mean_squared_error([s], [t], squared=False)
    rmses.append(rmse_score)

rmses = pd.DataFrame(rmses)
rmses.columns = ['rmse']
X_valid_withRMSE = pd.concat([X_valid_records, rmses], axis=1)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argumen

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001263 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2466
[LightGBM] [Info] Number of data points in the train set: 20140, number of used features: 14
[LightGBM] [Info] Start training from score 118627.765343
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 49671.4	valid_1's rmse: 44899.3
[20]	training's rmse: 35210.5	valid_1's rmse: 31638.5
[30]	training's rmse: 27066.2	valid_1's rmse: 24594
[40]	training's rmse: 21764	valid_1's rmse: 20994.9
[50]	training's rmse: 18130.5	valid_1's rmse: 19011.1
[60]	training's rmse: 15978.7	valid_1's rmse: 18020.1
[70]	training's rmse: 14648.5	valid_1's rmse: 17303.6
[80]	training's rmse: 13725.4	valid_1's rmse: 16820.5
[90]	training's rmse: 12931.2	valid_1's rmse: 16417.9
[100]	training's rmse: 12289.9	valid_1's rm

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argumen

[10]	training's rmse: 48710.8	valid_1's rmse: 48597.8
[20]	training's rmse: 34629.5	valid_1's rmse: 35393.1
[30]	training's rmse: 26650	valid_1's rmse: 28654.3
[40]	training's rmse: 21548.4	valid_1's rmse: 24295.5
[50]	training's rmse: 18091.3	valid_1's rmse: 21232
[60]	training's rmse: 16031.9	valid_1's rmse: 19616.2
[70]	training's rmse: 14791.2	valid_1's rmse: 18883.7
[80]	training's rmse: 13887.3	valid_1's rmse: 18375.2
[90]	training's rmse: 13168.2	valid_1's rmse: 18012.7
[100]	training's rmse: 12583.7	valid_1's rmse: 17906.3
[110]	training's rmse: 12068	valid_1's rmse: 17764
[120]	training's rmse: 11639.5	valid_1's rmse: 17705.9
[130]	training's rmse: 11261.1	valid_1's rmse: 17620.3
[140]	training's rmse: 10922.9	valid_1's rmse: 17569
[150]	training's rmse: 10639.9	valid_1's rmse: 17506.7
[160]	training's rmse: 10397.3	valid_1's rmse: 17527.5
Early stopping, best iteration is:
[150]	training's rmse: 10639.9	valid_1's rmse: 17506.7


/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argumen

...

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argumen

[10]	training's rmse: 47961	valid_1's rmse: 53090.4
[20]	training's rmse: 33743.3	valid_1's rmse: 40066.8
[30]	training's rmse: 25712.9	valid_1's rmse: 32899.5
[40]	training's rmse: 20459.1	valid_1's rmse: 28936.8
[50]	training's rmse: 17084.9	valid_1's rmse: 26920.6
[60]	training's rmse: 15072.2	valid_1's rmse: 25918.8
[70]	training's rmse: 13821.3	valid_1's rmse: 25154.2
[80]	training's rmse: 12902.4	valid_1's rmse: 24720.6
[90]	training's rmse: 12233	valid_1's rmse: 24487.8
[100]	training's rmse: 11671.3	valid_1's rmse: 24279.5
[110]	training's rmse: 11199.5	valid_1's rmse: 24134.6
[120]	training's rmse: 10808.8	valid_1's rmse: 24081.6
[130]	training's rmse: 10424.7	valid_1's rmse: 24036.5
[140]	training's rmse: 10074.7	valid_1's rmse: 23956.6
[150]	training's rmse: 9718.16	valid_1's rmse: 23900.8
[160]	training's rmse: 9405.74	valid_1's rmse: 23829.6
[170]	training's rmse: 9141.61	valid_1's rmse: 23848.3
[180]	training's rmse: 8889.89	valid_1's rmse: 23780.5
[190]	training's rmse: 

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argumen

[20]	training's rmse: 33908.8	valid_1's rmse: 39139.4
[30]	training's rmse: 25767.6	valid_1's rmse: 31651.8
[40]	training's rmse: 20646.3	valid_1's rmse: 27207.2
[50]	training's rmse: 17222.8	valid_1's rmse: 24659.1
[60]	training's rmse: 15175.6	valid_1's rmse: 23436.2
[70]	training's rmse: 13720.8	valid_1's rmse: 22828.8
[80]	training's rmse: 12745.9	valid_1's rmse: 22411.4
[90]	training's rmse: 12044.8	valid_1's rmse: 22127.7
[100]	training's rmse: 11500.7	valid_1's rmse: 21915.3
[110]	training's rmse: 11062.8	valid_1's rmse: 21859.2
[120]	training's rmse: 10670.8	valid_1's rmse: 21863.3
Early stopping, best iteration is:
[116]	training's rmse: 10818.5	valid_1's rmse: 21850.2


,importance
築年数,5.063084e+13
面積,3.730951e+14
契約期間,8.677085e+12
定期借家,1.944335e+12
所在階,7.109648e+12
全体の階数,2.551252e+13
最寄り駅,5.565025e+13
所要時間,4.502912e+12
平均地価,1.819959e+14
所在地,2.129531e+13


In [122]:
df_importance

,importance
面積,3.730951e+14
L,3.399677e+14
平均地価,1.819959e+14
最寄り駅,5.565025e+13
築年数,5.063084e+13
全体の階数,2.551252e+13
所在地,2.129531e+13
契約期間,8.677085e+12
所在階,7.109648e+12
所要時間,4.502912e+12


In [125]:
y_valid = pd.DataFrame(y_valid)
y_valid.columns = ['賃料']
DATA_valid_withRMSE = pd.concat([y_valid, X_valid_withRMSE], axis=1)

In [126]:
DATA_valid_withRMSE

,賃料,index,築年数,面積,契約期間,定期借家,所在階,全体の階数,最寄り駅,所要時間,平均地価,所在地,部屋数,L,D,S,rmse
0,77000,17249,125.0,20.16,2.0,1,1.0,5.0,339,10.0,3.643333e+05,336,1,0,0,0,3076.134017
1,90000,2953,1.0,25.50,2.0,1,2.0,5.0,70,3.0,3.660000e+05,215,1,0,0,0,3493.273640
2,68000,8326,389.0,33.00,2.0,1,2.0,2.0,32,15.0,4.520000e+05,186,1,1,1,0,9744.951814
3,170000,2921,101.0,40.00,NaN,3,7.0,7.0,23,2.0,1.408000e+06,2,1,1,1,0,4776.473364
4,138000,25487,31.0,25.54,2.0,1,4.0,5.0,78,6.0,3.151500e+06,214,1,0,0,0,14193.195763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6289,300000,3590,138.0,62.53,NaN,3,10.0,24.0,157,3.0,2.930000e+06,180,2,1,1,0,1792.221091
6290,65000,1180,365.0,22.91,2.0,1,3.0,4.0,96,3.0,7.557143e+05,269,1,0,0,0,4590.483093
6291,110500,28266,317.0,56.01,2.0,1,2.0,3.0,29,13.0,3.016667e+05,115,3,0,1,0,9686.189898
6292,80000,12674,89.0,22.98,NaN,3,4.0,5.0,24,13.0,5.166667e+05,24,1,0,0,0,2908.301916


In [128]:
DATA_valid_withRMSE.corr()['rmse']

賃料       0.605463
index   -0.017248
築年数      0.008996
面積       0.306256
契約期間     0.049220
定期借家     0.004624
所在階      0.094175
全体の階数    0.087757
最寄り駅     0.032924
所要時間    -0.013806
平均地価     0.106019
所在地      0.018558
部屋数      0.143940
L        0.132297
D        0.118579
S        0.123421
rmse     1.000000
Name: rmse, dtype: float64

### RMSEとの強い相関のある「賃料」について、どうするのかを考える

In [130]:
y_train = rent
y_train

,賃料
0,75000
1,76000
2,110000
3,150000
4,74000
...,...
31465,80000
31466,54500
31467,125000
31468,98000


In [142]:
print("賃料が40万以上の数：",len(y_train[y_train['賃料'] > 400000]))
print("賃料が50万以上の数：",len(y_train[y_train['賃料'] > 500000]))
print("賃料が60万以上の数：",len(y_train[y_train['賃料'] > 600000]))
print("賃料が70万以上の数：",len(y_train[y_train['賃料'] > 700000]))
print("賃料が80万以上の数：",len(y_train[y_train['賃料'] > 800000]))
print("賃料が90万以上の数：",len(y_train[y_train['賃料'] > 900000]))
print("賃料が100万以上の数：",len(y_train[y_train['賃料'] > 1000000]))


賃料が40万以上の数： 243
賃料が50万以上の数： 124
賃料が60万以上の数： 73
賃料が70万以上の数： 46
賃料が80万以上の数： 33
賃料が90万以上の数： 22
賃料が100万以上の数： 16


賃料が50万以上のデータを抜いて学習させてみる。

In [166]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, land_aveprice, addresses, room_arrange[["部屋数","L","D","S"]]], axis=1)
y_train = rent
all_train_DATA = pd.concat([y_train, X_train], axis=1)
all_train_DATA = all_train_DATA[all_train_DATA['賃料'] < 500000]

In [167]:
X_train = all_train_DATA.drop('賃料', axis=1)
y_train = all_train_DATA['賃料']

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_land_aveprice, test_addresses, test_room_arrange[["部屋数","L","D","S"]]], axis=1)
category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','S']

In [170]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
cv = KFold(n_splits=5, shuffle=True, random_state=0)

#これより交差検証のためにインデックスを振り直すので、最後にインデックスを元に戻すことに備えて元々のインデックスを退避させておく。
origin_index = X_train.index #origin_index = y_train.indexでも同じこと
X_train = X_train.reset_index()#交差検証のためにインデックス振り直し
X_train = X_train.drop('index', axis=1)##reset_index()をした結果、元のインデックスが"index"列として含まれてしまうので落としておく
y_train = y_train.reset_index()#交差検証のためにインデックス振り直し
y_train = y_train.drop('index',axis=1)##reset_index()をした結果、元のインデックスが"index"列として含まれてしまうので落としておく

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    params = {
        'objective':'regression',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda':  2.903886846508796e-05,
        'max_bin': 288,
        'num_leaves': 116
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=1000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)
# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
df_importance = pd.DataFrame(importance)
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)


/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argumen

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001451 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2146
[LightGBM] [Info] Number of data points in the train set: 20055, number of used features: 14
[LightGBM] [Info] Start training from score 115551.405086
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 38381.6	valid_1's rmse: 38884.3
[20]	training's rmse: 26378	valid_1's rmse: 27711
[30]	training's rmse: 19471.4	valid_1's rmse: 21639.6
[40]	training's rmse: 15359.4	valid_1's rmse: 18223.6
[50]	training's rmse: 12939.8	valid_1's rmse: 16401.1
[60]	training's rmse: 11364.9	valid_1's rmse: 15204.9
[70]	training's rmse: 10342.7	valid_1's rmse: 14471.9
[80]	training's rmse: 9557.93	valid_1's rmse: 13978.8
[90]	training's rmse: 8952.34	valid_1's rmse: 13641.3
[100]	training's rmse: 8495.71	valid_1's rm

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argumen

[20]	training's rmse: 26407	valid_1's rmse: 27688.1
[30]	training's rmse: 19449.9	valid_1's rmse: 21381.4
[40]	training's rmse: 15369.1	valid_1's rmse: 17823.6
[50]	training's rmse: 12919.1	valid_1's rmse: 15854.9
[60]	training's rmse: 11337.4	valid_1's rmse: 14666.9
[70]	training's rmse: 10304.8	valid_1's rmse: 13964
[80]	training's rmse: 9543.25	valid_1's rmse: 13474.9
[90]	training's rmse: 8944.86	valid_1's rmse: 13154.3
[100]	training's rmse: 8487.22	valid_1's rmse: 12911.3
[110]	training's rmse: 8112.14	valid_1's rmse: 12758.6
[120]	training's rmse: 7796.41	valid_1's rmse: 12621.3
[130]	training's rmse: 7525.3	valid_1's rmse: 12528.1
[140]	training's rmse: 7280.77	valid_1's rmse: 12450.9
[150]	training's rmse: 7040.57	valid_1's rmse: 12388.9
[160]	training's rmse: 6827.41	valid_1's rmse: 12340.8
[170]	training's rmse: 6644.61	valid_1's rmse: 12305.3
[180]	training's rmse: 6484.1	valid_1's rmse: 12262.4
[190]	training's rmse: 6321.79	valid_1's rmse: 12224.4
[200]	training's rmse: 6

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argumen

...

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argumen

[20]	training's rmse: 26497.2	valid_1's rmse: 27176.2
[30]	training's rmse: 19549	valid_1's rmse: 20759.2
[40]	training's rmse: 15444.9	valid_1's rmse: 17290.9
[50]	training's rmse: 13011.9	valid_1's rmse: 15393
[60]	training's rmse: 11468.8	valid_1's rmse: 14269.5
[70]	training's rmse: 10427.1	valid_1's rmse: 13608
[80]	training's rmse: 9657.77	valid_1's rmse: 13178.1
[90]	training's rmse: 9068.04	valid_1's rmse: 12871.8
[100]	training's rmse: 8596.21	valid_1's rmse: 12674.3
[110]	training's rmse: 8214.88	valid_1's rmse: 12505.7
[120]	training's rmse: 7908.58	valid_1's rmse: 12399.8
[130]	training's rmse: 7638.4	valid_1's rmse: 12303.3
[140]	training's rmse: 7383.79	valid_1's rmse: 12215
[150]	training's rmse: 7153.13	valid_1's rmse: 12138.7
[160]	training's rmse: 6934.38	valid_1's rmse: 12089.6
[170]	training's rmse: 6752.87	valid_1's rmse: 12041.6
[180]	training's rmse: 6583.95	valid_1's rmse: 11992.7
[190]	training's rmse: 6437.27	valid_1's rmse: 11956.7
[200]	training's rmse: 6295

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argumen

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001084 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2149
[LightGBM] [Info] Number of data points in the train set: 20056, number of used features: 14
[LightGBM] [Info] Start training from score 115769.414190
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 38220.2	valid_1's rmse: 39189.4
[20]	training's rmse: 26273.8	valid_1's rmse: 27782.3
[30]	training's rmse: 19407.1	valid_1's rmse: 21639
[40]	training's rmse: 15298.7	valid_1's rmse: 18161.1
[50]	training's rmse: 12895.4	valid_1's rmse: 16250.6
[60]	training's rmse: 11366.6	valid_1's rmse: 15104.6
[70]	training's rmse: 10374.8	valid_1's rmse: 14406.8
[80]	training's rmse: 9617.71	valid_1's rmse: 13924.6
[90]	training's rmse: 9047.63	valid_1's rmse: 13596
[100]	training's rmse: 8586.31	valid_1's rm

,importance
L,2.795217e+14
面積,1.803161e+14
平均地価,9.911984e+13
最寄り駅,4.573870e+13
築年数,4.157081e+13
全体の階数,2.637997e+13
所在地,2.132395e+13
所在階,6.686167e+12
所要時間,5.302406e+12
部屋数,9.555254e+11
